In [1]:
import folium
import pandas as pd
import matplotlib.colors as mcolors
from matplotlib import colormaps
import branca.colormap as cm

# Load CSV Data
df = pd.read_csv("D:/143/project/i5_2019_coordinates.csv")  #Make sure the CSV contains 'latitude', 'longitude', and 'AADT' columns
df["Longitude"] = -df["Longitude"] # Convert longitude to negative values if needed
# Normalize AADT data for color mapping
norm = mcolors.Normalize(vmin=df["VEHICLE_AADT_TOTAL"].min(), vmax=df["VEHICLE_AADT_TOTAL"].max())
colormap = colormaps.get_cmap("YlOrRd")

In [2]:
# Create a Folium map centered on the average coordinates
m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=7, tiles="OpenStreetMap")

# Draw lines sequentially and change the color based on AADT values
for i in range(len(df) - 1):
    lat1, lon1, aadt1 = df.iloc[i][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]
    lat2, lon2, aadt2 = df.iloc[i + 1][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]

    # Get the color based on the average AADT between two points
    color = mcolors.to_hex(colormap(norm((aadt1 + aadt2) / 2)))  # 取两个点的 AADT 平均值计算颜色

    # Draw the line
    folium.PolyLine(
        locations=[(lat1, lon1), (lat2, lon2)],
        color=color,
        weight=5,
        opacity=0.8
    ).add_to(m)

display(m)


In [3]:
import folium
import branca.colormap as cm
import numpy as np

# 1️ Create a Folium map
m = folium.Map(
    location=[df["Latitude"].mean(), df["Longitude"].mean()], 
    zoom_start=7, 
    tiles="OpenStreetMap"
)

# 2️ Color mapping (used for gradient lines and legend)
colormap = cm.LinearColormap(
    colors=["blue", "green", "yellow", "red"],  # Color gradient
    vmin=df["VEHICLE_AADT_TOTAL"].min(),  
    vmax=df["VEHICLE_AADT_TOTAL"].max()
)

# 3️ Draw smooth gradient lines
num_subsegments = 10  # Split each line segment into 10 parts to make the color transition smoother
for i in range(len(df) - 1):
    lat1, lon1, aadt1 = df.iloc[i][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]
    lat2, lon2, aadt2 = df.iloc[i + 1][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]

    # Generate interpolated latitudes and longitudes (subdivide the line)
    lats = np.linspace(lat1, lat2, num_subsegments)
    lons = np.linspace(lon1, lon2, num_subsegments)
    aadts = np.linspace(aadt1, aadt2, num_subsegments)  # Interpolate AADT as well for smooth color transition

    # Draw smaller line segments with color gradients
    for j in range(num_subsegments - 1):
        color = colormap(aadts[j])  # Calculate the color for the current small segment


        folium.PolyLine(
            locations=[(lats[j], lons[j]), (lats[j + 1], lons[j + 1])],
            color=color,
            weight=5,
            opacity=0.8
        ).add_to(m)


# 4 Add AADT color legend
colormap.caption = "AADT Traffic Data"
colormap.add_to(m)

# 5 Display the map
display(m)
